In [ ]:
!pip install selenium
!pip install msedge-selenium-tools
!pip install bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.6/904.6 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: selenium
    Found existing installation: selenium 4.31.0
    Uninstalling selenium-4.31.0:
      Successfully uninstalled selenium-4.31.0


In [ ]:
!pip install webdriver-manager


In [ ]:
pip install -U selenium


  Using cached selenium-4.31.0-py3-none-any.whl.metadata (7.5 kB)
Using cached selenium-4.31.0-py3-none-any.whl (9.4 MB)
  Attempting uninstall: selenium
    Found existing installation: selenium 3.141.0
    Uninstalling selenium-3.141.0:
      Successfully uninstalled selenium-3.141.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
msedge-selenium-tools 3.141.4 requires selenium==3.141, but you have selenium 4.31.0 which is incompatible.


In [ ]:
driver = webdriver.chrome

In [ ]:
url = 'https://www.amazon.com/'
driver.get(url)

AttributeError: module 'selenium.webdriver.chrome' has no attribute 'get'

In [ ]:
def my_url(keyword):
    temp = 'https://www.amazon.com/s?k={}&crid=2BL65YW8IH6D6&sprefix=phone+cas%2Caps%2C234&ref=nb_sb_noss_2'
    keyword = keyword.replace(' ','+')
    return temp.format(keyword)


In [ ]:
url = my_url('laptop charger')
url

'https://www.amazon.com/s?k=laptop+charger&crid=2BL65YW8IH6D6&sprefix=phone+cas%2Caps%2C234&ref=nb_sb_noss_2'

In [ ]:
url = my_url('kids toys')
url

'https://www.amazon.com/s?k=kids+toys&crid=2BL65YW8IH6D6&sprefix=phone+cas%2Caps%2C234&ref=nb_sb_noss_2'

In [ ]:
url = my_url('educational insights')
url

'https://www.amazon.com/s?k=educational+insights&crid=2BL65YW8IH6D6&sprefix=phone+cas%2Caps%2C234&ref=nb_sb_noss_2'

In [ ]:
!wget -O chromedriver.zip https://chromedriver.storage.googleapis.com/136.0.7326.0/chromedriver_linux64.zip


--2025-05-02 06:55:19--  https://chromedriver.storage.googleapis.com/136.0.7326.0/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 142.250.157.207, 142.251.8.207, 142.251.170.207, ...
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|142.250.157.207|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-05-02 06:55:20 ERROR 404: Not Found.



In [ ]:
pip install selenium


In [ ]:
!rm /usr/bin/chromedriver


In [ ]:
!wget -q -O /tmp/chromedriver.zip https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.39/linux64/chromedriver-linux64.zip


In [ ]:
!unzip /tmp/chromedriver.zip -d /tmp/
!mv /tmp/chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver


Archive:  /tmp/chromedriver.zip
  inflating: /tmp/chromedriver-linux64/LICENSE.chromedriver  
  inflating: /tmp/chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflating: /tmp/chromedriver-linux64/chromedriver  


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver"), options=chrome_options)
driver.get("https://www.google.com")
print(driver.title)
driver.quit()


Google


In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from google.colab import files

# URL construction function
def my_url(keyword):
    temp = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    keyword = keyword.replace(' ', '+')
    url = temp.format(keyword)
    url += '&page{}'  # Add page query placeholder
    return url

# Function to extract records from each item
def extract_record(obj):
    # Debugging: Print the obj to verify structure
    print(f"Processing item: {obj}")

    # Safely extract the <a> tag inside <h2> (checking if it exists)
    atag = obj.h2.a if obj.h2 else None
    if not atag:
        return None  # Skip if there's no <a> tag inside <h2>

    description = atag.text.strip() if atag else 'No description'
    url = 'https://www.amazon.com' + atag.get('href') if atag else 'No URL'

    # Try to get the price, if not found, return None (skip this item)
    try:
        parent = obj.find('span', 'a-price')
        price = parent.find('span', 'a-offscreen').text if parent else 'No price'
    except AttributeError:
        price = 'No price'

    # Try to get the rating and review count, if not found, assign empty values
    try:
        rate = obj.i.text if obj.i else 'No rating'
        counts_review = obj.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text if obj.find('span', {'class': 'a-size-base', 'dir': 'auto'}) else 'No reviews'
    except AttributeError:
        rate = 'No rating'
        counts_review = 'No reviews'

    # Try to get the image URL
    image = obj.find('img', {'class': 's-image'}).get('src') if obj.find('img', {'class': 's-image'}) else 'No image'

    # Return the extracted result as a tuple
    result = (description, price, rate, counts_review, url, image)
    return result

# Main function to perform web scraping
def main(keyword):
    # Chrome WebDriver options setup
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode (no GUI)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Start the WebDriver with the specified options
    driver = webdriver.Chrome(options=chrome_options)
    records = []  # Initialize the records list
    url = my_url(keyword)

    # Loop through pages (example, 1-3 pages)
    for page in range(1, 4):  # Adjust the range to scrape more pages if necessary
        driver.get(url.format(page))

        # Wait explicitly for search result items to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[data-component-type="s-search-result"]'))
        )

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Debugging: Print out the page source to check if data is being loaded
        print(f"Scraping page {page}")

        # Extract search result containers
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        if results:
            print(f"Found {len(results)} results on page {page}")

        # Extract records
        for item in results:
            record = extract_record(item)
            if record:
                print(f"Extracted record: {record}")  # Debugging extracted record
                records.append(record)  # Add record to the list

    # Save results to CSV
    with open('/content/Results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'Reviews Count', 'URL', 'Image link'])
        writer.writerows(records)

    # Trigger download of the CSV file
    files.download('/content/Results.csv')

    # Quit the driver session after scraping is done
    driver.quit()

# Run the main function with a keyword
main('educational insights')


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
chrome_options.add_argument("--headless")


In [ ]:
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")


In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from google.colab import files
import time

# Create Amazon URL from keyword
def my_url(keyword):
    base = 'https://www.amazon.com/s?k={}&page={}'
    keyword = keyword.replace(' ', '+')
    return base.format(keyword, '{}')

# Extract single product record
def extract_record(obj):
    try:
        atag = obj.h2.a
        description = atag.text.strip()
        url = 'https://www.amazon.com' + atag.get('href')
    except:
        return None

    try:
        price = obj.select_one("span.a-price > span.a-offscreen").text.strip()
    except:
        price = 'No price'

    try:
        rate = obj.select_one("span.a-icon-alt").text.strip()
    except:
        rate = 'No rating'

    try:
        reviews = obj.select_one("span.a-size-base.s-underline-text").text.strip()
    except:
        reviews = 'No reviews'

    try:
        image = obj.find('img', {'class': 's-image'}).get('src')
    except:
        image = 'No image'

    return (description, price, rate, reviews, url, image)

# Main scraping function
def main(keyword):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    records = []
    for page in range(1, 4):
        print(f"Scraping page {page}")
        driver.get(my_url(keyword).format(page))
        time.sleep(2)  # allow JS to render
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        print(f"Found {len(results)} results on page {page}")

        for item in results:
            record = extract_record(item)
            if record:
                print(f"Extracted record: {record[0]}")
                records.append(record)

    driver.quit()

    # Save to CSV
    filename = '/content/Results.csv'
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'Reviews Count', 'URL', 'Image link'])
        writer.writerows(records)

    files.download(filename)

# Run the script
main('educational insights')


Scraping page 1
Found 48 results on page 1
Scraping page 2
Found 48 results on page 2
Scraping page 3
Found 48 results on page 3


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}

def get_url(keyword, page=1):
    return f"https://www.amazon.com/s?k={keyword.replace(' ', '+')}&page={page}"

def extract_product_info(item):
    try:
        title = item.h2.text.strip()
        link = "https://www.amazon.com" + item.h2.a['href']
    except AttributeError:
        return None

    try:
        price = item.find('span', 'a-offscreen').text.strip()
    except:
        price = 'N/A'

    try:
        rating = item.i.text.strip()
    except:
        rating = 'N/A'

    try:
        reviews = item.find('span', {'class': 'a-size-base'}).text.strip()
    except:
        reviews = 'N/A'

    return (title, price, rating, reviews, link)

def scrape_amazon(keyword, pages=3):
    results = []
    for page in range(1, pages + 1):
        print(f"Scraping page {page}")
        url = get_url(keyword, page)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        items = soup.find_all('div', {'data-component-type': 's-search-result'})
        print(f"Found {len(items)} items")

        for item in items:
            product = extract_product_info(item)
            if product:
                results.append(product)

        time.sleep(random.uniform(1.5, 3.5))  # Be polite

    return results

def save_to_csv(data, filename='results.csv'):
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Title', 'Price', 'Rating', 'Review Count', 'URL'])
        writer.writerows(data)
    print(f"Saved {len(data)} items to {filename}")

# Run it
keyword = 'educational insights'
data = scrape_amazon(keyword)
save_to_csv(data)


Scraping page 1
Found 0 items
Scraping page 2
Found 0 items
Scraping page 3
Found 0 items
Saved 0 items to results.csv


In [77]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

def my_url(keyword):
    base = 'https://www.amazon.com/s?k={}&page={}'
    return base.format(keyword.replace(' ', '+'), '{}')  # placeholder for page number

def extract_record(item):
    try:
        atag = item.h2.a
        description = atag.text.strip()
        url = 'https://www.amazon.com' + atag.get('href')
    except AttributeError:
        return None

    try:
        price = item.find('span', 'a-price').find('span', 'a-offscreen').text
    except AttributeError:
        price = 'N/A'

    try:
        rate = item.i.text.strip()
    except AttributeError:
        rate = 'N/A'

    try:
        reviews = item.find('span', {'class': 'a-size-base'}).text.strip()
    except AttributeError:
        reviews = 'N/A'

    try:
        image = item.find('img', {'class': 's-image'})['src']
    except (AttributeError, TypeError):
        image = 'N/A'

    return (description, price, rate, reviews, url, image)

def main(keyword):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(), options=options)
    records = []

    for page in range(1, 4):  # Scrape pages 1 to 3
        print(f"Scraping page {page}")
        driver.get(my_url(keyword).format(page))
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        print(f"Found {len(results)} results")

        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)

    driver.quit()

    # Write to CSV
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'Reviews', 'URL', 'Image'])
        writer.writerows(records)

    print(f"Scraping completed. {len(records)} products saved to results.csv.")

# Run the scraper
main('educational insights')


Scraping page 1
Found 0 results
Scraping page 2
Found 0 results
Scraping page 3
Found 0 results
Scraping completed. 0 products saved to results.csv.
